In [70]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
from google.colab import drive
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
drive.mount('/content/drive')
dataset_path = "/content/drive/MyDrive/Trade_finance_small_dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
model = torchvision.models.efficientnet_v2_s(progress = True, weights = "DEFAULT")

In [72]:
num_classes = 5
model.classifier = nn.Linear(1280, num_classes)

In [89]:
classes = os.listdir(dataset_path)
classes
label_dict = {value: idx for idx, value in enumerate(classes)}
id_label_dict = {idx: value for idx, value in enumerate(classes)}

In [90]:
id_label_dict

{0: 'packing_list',
 1: 'bill_of_lading_first_page',
 2: 'certificate_of_origin_first_page',
 3: 'Insurance_Certificate_pngs_first_page',
 4: 'covering_schedule'}

In [74]:
train_images_dict = {
            'images': [],  # List of train images as PIL images
            'labels': []  # List of corresponding labels (strings) for train images
        }

test_images_dict = {
            'images': [],
            'labels': []
        }



In [75]:
cd drive/MyDrive

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


In [76]:
os.mkdir("train_images")
os.mkdir("test_images")

In [77]:
import random
import shutil
from PIL import Image
random.seed(10)
train_files = []
test_files = []
for document in os.listdir(dataset_path):
  document_path = os.path.join(dataset_path,document)
  files= os.listdir(os.path.join(dataset_path,document))
  random.shuffle(files)
  for train_file in files[:-int(0.2 * len(files))]:
    if train_file not in os.listdir("train_images"):
      shutil.copy(os.path.join(document_path, train_file), "train_images")
      img = Image.open(os.path.join(document_path, train_file)).convert("RGB")
      train_images_dict["images"].append(img)
      train_images_dict["labels"].append(label_dict[document])
  for test_file in files[-int(0.2 * len(files)):]:
    if test_file not in os.listdir("test_images"):
      shutil.copy(os.path.join(document_path, test_file), "test_images")
      img = Image.open(os.path.join(document_path, test_file)).convert("RGB")
      test_images_dict["images"].append(img)
      test_images_dict["labels"].append(label_dict[document])


In [79]:
class ImageDataset(Dataset):
    '''
        Sample Input:
        train_images = {
            'images': [PIL_image1, PIL_image2, PIL_image3, ...],  # List of train images as PIL images
            'labels': ['cat', 'dog', 'bird', ...]  # List of corresponding labels (strings) for train images
        }

        test_images = {
            'images': [PIL_image4, PIL_image5, PIL_image6, ...],
            'labels': ['cat', 'dog', 'bird', ...]
        }

        val_images = {
            'images': [PIL_image7, PIL_image8, PIL_image9, ...],
            'labels': ['cat', 'dog', 'bird', ...]
        }
    '''
    def __init__(self, image_dict, transform=None):
        self.image_dict = image_dict
        self.transform = transform

    def __len__(self):
        return len(self.image_dict['images'])

    def __getitem__(self, idx):
        image = self.image_dict['images'][idx]
        label = self.image_dict['labels'][idx]

        if self.transform:
            image = self.transform(image)

        # Convert label to tensor
        label = torch.tensor(int(label))

        return image, label

In [80]:
label = 1
a = torch.tensor(int(label))


In [81]:
class ImageTransformer:
    def __init__(self):
        self._image_size = (224, 224)

        self.transform = transforms.Compose([
            transforms.Resize(self._image_size),
            transforms.ToTensor(),
            # transforms.Normalize(mean=self._mean, std=self._std)
        ])

In [82]:
# Create the data loaders for train, test, and validation
train_dataset = ImageDataset(train_images_dict, transform=ImageTransformer().transform)
test_dataset = ImageDataset(test_images_dict, transform=ImageTransformer().transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [84]:
import torch.nn as nn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [85]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
      #print(data)
      # Every data instance is an input + label pair
      inputs, labels = data
      print("input is ", inputs[0].shape)

      # Zero your gradients for every batch!
      optimizer.zero_grad()

      # Make predictions for this batch
      outputs = model(inputs)
      print(outputs[0].shape)

      # Compute the loss and its gradients
      loss = loss_fn(outputs, labels)
      loss.backward()

      # Adjust learning weights
      optimizer.step()

      # Gather data and report
      running_loss += loss.item()
    last_loss = running_loss /10    # loss per batch

    return last_loss

In [86]:
# Initializing in a separate cell so we can easily add more epochs to the same run
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))


In [87]:
EPOCHS = 10
epoch_number = 0
best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    # We don't need gradients on to do reporting
    model.train(False)

    running_test_loss = 0.0
    for i, test_data in enumerate(test_loader):
        test_inputs, test_labels = test_data
        test_outputs = model(test_inputs)
        test_loss = loss_fn(test_outputs, test_labels)
        running_test_loss += test_loss

    avg_test_loss = running_test_loss / (i + 1)
    print('LOSS train {} test {}'.format(avg_loss, avg_test_loss))

    # Log the running loss averaged per batch
    # for both training and validation

    # Track best performance, and save the model's state
    # if avg_test_loss < best_vloss:
    #     best_vloss = avg_test_loss
    #     model_path = 'model_{}_{}'.format(timestamp, epoch_number)
    #     torch.save(model.state_dict(), model_path)

    epoch_number += 1
    if epoch_number == 5 or epoch_number == 10:
      path = "efficient_net_models/model_{}epochs".format(epoch_number)
      torch.save(model, path)

EPOCH 1:
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
LOSS train 1.5908841967582703 test 2.9817464351654053
EPOCH 2:
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Size([5])
input is  torch.Size([3, 224, 224])
torch.Si